In [1]:
from mmpose.apis import MMPoseInferencer
from datetime import datetime
import subprocess
import os
import json
import glob
import cv2

# Only include the following code if you are running the complete cloned project
while os.getcwd().split('/')[-1] != 'mmpose-synthetic-tune':
    os.chdir('..')

/opt/conda/lib/python3.11/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [2]:
coco_cow_category = {
    "id": 1,
    "name": "cow",
    "supercategory": "",
    "keypoints": [
        "Back1",
        "Back2",
        "Back3",
        "Back4",
        "Head",
        "Nose",
        "Neck",
        "L_Shoulder",
        "L_Elbow",
        "L_F_Paw",
        "R_Shoulder",
        "R_Elbow",
        "R_F_Paw",
        "Belly",
        "L_Hip",
        "L_Knee",
        "L_H_Paw",
        "R_Hip",
        "R_Knee",
        "R_H_Paw"
    ],
    "skeleton": [
        [
            3,
            4
        ],
        [
            16,
            17
        ],
        [
            12,
            13
        ],
        [
            8,
            9
        ],
        [
            11,
            14
        ],
        [
            18,
            1
        ],
        [
            6,
            5
        ],
        [
            18,
            19
        ],
        [
            14,
            18
        ],
        [
            14,
            15
        ],
        [
            9,
            10
        ],
        [
            8,
            14
        ],
        [
            1,
            2
        ],
        [
            19,
            20
        ],
        [
            15,
            1
        ],
        [
            15,
            16
        ],
        [
            7,
            6
        ],
        [
            4,
            7
        ],
        [
            11,
            12
        ],
        [
            2,
            3
        ],
        [
            7,
            11
        ],
        [
            7,
            8
        ]
    ]
    }

In [3]:
class MMPoseModelCoach:
    command = 'python'
    train_script = 'mmpose/tools/train.py'
    test_script = 'mmpose/tools/test.py'
    
    detector_model = {  #rtmdet
        "det_model": 'mmdetection/configs/rtmdet/rtmdet_l_swin_b_p6_4xb16-100e_coco.py',
        "det_weights": 'checkpoints/rtmdet_l_swin_b_p6_4xb16-100e_coco-a1486b6f.pth'
    }

    def __init__(self, config=None, resume=True, work_dir=None, notes=''):
        self.creation_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        self.config = config
        self.resume = '--resume' if resume == True else ''
        if work_dir is not None:
            self.work_dir = work_dir
            self.config_path = glob.glob(f'{self.work_dir}/*.py')[0]
        else:
            self.config_path = f'custom-configs/{self.config}'
            self.work_dir = f'models/_train-{self.creation_time}-{notes}'

        self.train_args = [
            self.command,
            self.train_script,
            self.config_path,
            '--work-dir',
            self.work_dir,
            self.resume,
        ]

    def train(self):
        subprocess.run(self.train_args)

    # def test(self, chkpoint):
    #     test_args = [
    #         self.command,
    #         self.test_script,
    #         self.config_path,
    #         f"{self.work_dir}/{chkpoint}",
    #         '--show-dir',
    #         'test_result'
    #     ]
    #     subprocess.run(test_args)

    def visualize_results(self, vis_input, model_ckpt=None, radius=4, thickness=1):
        if model_ckpt is None:
            checkpoint_path = glob.glob(f'{self.work_dir}/best*.pth')[0]
        else:
            checkpoint_path = f'{self.work_dir}/{model_ckpt}'
        
        poser_model = {
            "pose2d": self.config_path,
            "pose2d_weights": checkpoint_path
        }

        inferencer = MMPoseInferencer(**poser_model, **self.detector_model, device='cuda:0')

        self.input_path = vis_input
        self.output_path = f'{self.work_dir}/vis_results'

        result_generator = inferencer(
            self.input_path,
            radius=radius,
            thickness=thickness,
            vis_out_dir=self.output_path,
            draw_heatmap=True,
            det_cat_ids=5
        )

        self.results = [res for res in result_generator]
        return self.results
        
    def results_to_coco(self):
        if not hasattr(self, 'results'):
            print('Run visualize_results first')
            return
        
        # Creating images
        all_images = []
        if os.path.isdir(self.input_path):
            file_names = sorted(glob.glob(f'{self.input_path}/*'))
        else:
            file_names = [self.input_path]
            
        id = 0
        for file_name in file_names:
            img = cv2.imread(file_name)
            h, w = img.shape[:2]
            img_coco = {
                "id": id,
                "width": w,
                "height": h,
                "file_name": file_name.split('/')[-1],
                "license": 0,
                "flickr_url": "",
                "coco_url": "",
                "date_captured": 0
            }
            all_images.append(img_coco)
            id += 1
            
        
        # Creating annotations
        all_anns = []
        kp_groups = list(map(lambda r: r['predictions'][0][0]['keypoints'], self.results))
        kp_score_groups = list(map(lambda r: r['predictions'][0][0]['keypoint_scores'], self.results))
        id = 0
        for kps, scores in zip(kp_groups, kp_score_groups):
            # base for coco keypoint annotations
            ann_coco = {
                "id": id,
                "image_id": id,
                "category_id": 1,
                "segmentation": [],
                "area": 0,
                "bbox": [],
                "iscrowd": 0,
                "attributes": {
                    "occluded": False,
                    "keyframe": False
                },
                "keypoints": [],
                "num_keypoints": 0
            }
            
            # adding keypoints in coco format
            ann_kps = []
            visible_kps = []
            for kp, score in zip(kps, scores):
                visibility = 2 if score >= .3 else 0
                ann_kps.extend([*kp, visibility])
                if visibility == 2:
                    visible_kps.extend(kp)

            ann_coco['keypoints'] = ann_kps
            
            # Calculating bounding box
            visible_kp_xs = [visible_kps[i] for i in range(0, len(visible_kps), 2)]
            visible_kp_ys = [visible_kps[i] for i in range(1, len(visible_kps), 2)]
            bbox_x = min(visible_kp_xs)
            bbox_y = min(visible_kp_ys)
            bbox_w = max(visible_kp_xs) - bbox_x
            bbox_h = max(visible_kp_ys) - bbox_y
            ann_coco['bbox'] = [ bbox_x, bbox_y, bbox_w, bbox_h ]
            
            # Calculating area
            ann_coco['area'] = bbox_w * bbox_h
            all_anns.append(ann_coco)
            id += 1
            
        coco_dataset = {
            "images": all_images,
            "annotations": all_anns,
            "categories": [coco_cow_category]
        }
        
        with open(f'{self.output_path}/results_coco_{self.creation_time}.json', 'w') as f:
            json.dump(coco_dataset, f)
        
        return coco_dataset
        
        

In [4]:
def test_model(work_dir_pattern, ckpt_pattern, idx=''):
    cfg = glob.glob(f'models/{work_dir_pattern}/*.py')[0]
    ckpt = glob.glob(f'models/{work_dir_pattern}/{ckpt_pattern}')[0]

    test_command = [
        'python',
        'mmpose/tools/test.py',
        cfg,
        ckpt,
        '--show-dir',
        'results'
    ]

    print(' '.join(test_command))

    subprocess.run(test_command)
    ann_file = glob.glob('*.keypoints.json')[0]
    os.rename(ann_file, str(idx) + '-' + ckpt.split('/')[-1].replace('.pth', '') + '-' + ann_file.replace('.keypoints.json', '.json'))

In [5]:
def test_model_all_ckpts(work_dir_pattern, idx=''):
    cfg = glob.glob(f'models/{work_dir_pattern}/*.py')[0]
    ckpts = glob.glob(f'models/{work_dir_pattern}/epoch_*.pth')

    for ckpt in sorted(ckpts):
        test_command = [
            'python',
            'mmpose/tools/test.py',
            cfg,
            ckpt,
            '--show-dir',
            'results'
        ]

        print(' '.join(test_command))

        subprocess.run(test_command)
        ann_file = glob.glob('*.keypoints.json')[0]
        os.rename(ann_file, str(idx) + '-' + ckpt.split('/')[-1].replace('.pth', '') + '-' + ann_file.replace('.keypoints.json', '.json'))

### Train

### Train and test models for averaging

In [16]:
# for i in range(5):
#     real = MMPoseModelCoach(
#         config=f'20kp-real-simillar-fold-{i}.py',
#         notes=f'real-simillar-fold-{i}'
#     )
#     real.train()
#     test_model(f'*real-simillar-fold-{i}', 'best*', idx=i)

# for i in range(5):
#     synthetic = MMPoseModelCoach(
#         config=f'20kp-synthetic-varied-fold-{i}.py',
#         notes=f'synthetic-varied-fold-{i}'
#     )
#     synthetic.train()
#     test_model(f'*synthetic-varied-fold-{i}', 'best*', idx=i)

for i in range(10):
    combined = MMPoseModelCoach(
        config=f'20kp-synthetic-on-real-fold-{i}.py',
        notes=f'synthetic-on-real-fold-{i}'
    )
    combined.train()
    test_model(f'*synthetic-on-real-fold-{i}', 'best*', idx=i)

/opt/conda/lib/python3.11/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \
Traceback (most recent call last):
  File "/workspace/mmpose-synthetic-tune/mmpose/tools/train.py", line 162, in <module>
    main()
  File "/workspace/mmpose-synthetic-tune/mmpose/tools/train.py", line 144, in main
    cfg = Config.fromfile(args.config)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/mmengine/config/config.py", line 460, in fromfile
    lazy_import is None and not Config._is_lazy_import(filename):
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/mmengine/config/config.py", line 1661, in _is_lazy_import
    with open(filename, encoding='utf-8') as f:
         ^^

IndexError: list index out of range

### Visualizing

In [ ]:
_20kp.visualize_results(
    vis_input='/home/galiold/Desktop/istockphoto-104783196-612x612.jpg',
    thickness=2,
    radius=7
)

In [ ]:
_20kp_on_ap10k = MMPoseModelCoach(
    work_dir='models/_train-2024-04-24_13-48-52-20kp-train-on-ap10k-w48-adjusted-labels'
)

In [ ]:
_20kp_on_ap10k.test('best_coco_AP_epoch_110.pth')

### Visualize Only

In [ ]:
vis_input = '/home/galiold/projects/datasets/test-footage/rendered'

In [ ]:
_20kp_on_ap10k = MMPoseModelCoach(
    work_dir='models/trained-20kp-on-ap10k'
)
_20kp_on_ap10k.visualize_results(
    # model_ckpt='epoch_210.pth',
    vis_input=vis_input
)
dataset = _20kp_on_ap10k.results_to_coco()

In [ ]:
pretrained_ap10k = MMPoseModelCoach(
    work_dir='models/pretrained-hrnet_w48_ap10k_256x256-d95ab412_20211029'
)
pretrained_ap10k.visualize_results(
    model_ckpt='hrnet_w48_ap10k_256x256-d95ab412_20211029.pth',
    vis_input=vis_input
)